# Fast python loops with cython

Cython is essentially a Python to C translator. Cython allows you to use syntax similar to Python, while achieving speeds near that of C. 

This post describes how to use Cython to speed up a single Python function involving ‘tight loops’. I’ll leave more complicated applications - with many functions and classes - for a later post.

# Should I use Cython?

If you’re using Python and need performance there are a variety of options, see quantecon for a detailed comparison. And of course you could always choose a different language like Julia, or be brave and learn C itself.

While the static compilation approach of Cython may not be cutting edge, Cython is mature, well documented and capable of handling large complicated projects. Cython code lies behind many of the big Python scientific libraries including scikit-learn and pandas.

# The example

Our example function evaluates a Radial Basis Function (RBF) approximation scheme. We assume each data point is a ‘center’ and use Gaussian type RBFs

$$Y_{i}=\sum_{j=0}^{N} \beta_{j}~ e^{- ~\theta ~||~X_{i}-X_{j} ~||^{2} }$$

so our function takes an input data array X of shape (N, D), a parameter array $\beta $ of length N and a ‘bandwidth’ parameter $\theta $ and return an array of values Y of length N.

In [28]:
%load_ext cython

The cython extension is already loaded. To reload it, use:
  %reload_ext cython


In [29]:
import sys
import Cython
print("Python %d.%d.%d %s %s" % sys.version_info)
print("Cython %s" % Cython.__version__)

Python 3.5.1 final 0
Cython 0.23.4


# Python loops

Here’s the naive Python implementation

In [30]:
from math import exp
import numpy as np

def rbf_network(X, beta, theta):

    N = X.shape[0]
    D = X.shape[1]
    Y = np.zeros(N)

    for i in range(N):
        for j in range(N):
            r = 0
            for d in range(D):
                r += (X[j, d] - X[i, d]) ** 2
            r = r**0.5
            Y[i] += beta[j] * exp(-(r * theta)**2)

    return Y

Let’s make up some data



In [31]:
import numpy as np
D = 5
N = 1000
X = np.array([np.random.rand(N) for d in range(D)]).T
beta = np.random.rand(N)
theta = 10

Timing this in IPython we get



In [32]:
%timeit rbf_network(X, beta, theta)

1 loop, best of 3: 4.57 s per loop


Dam those Python loops are slow!



# scipy.interpolate.Rbf

So in this case we’re lucky and there’s an external numpy based implementation

In [33]:
from scipy.interpolate import Rbf
rbf = Rbf(X[:,0], X[:,1], X[:,2], X[:,3], X[:, 4], np.random.rand(N))
Xtuple = tuple([X[:, i] for i in range(D)])

In [34]:
%timeit rbf(Xtuple)


1 loop, best of 3: 185 ms per loop


Much better. But what if we want to go faster or we don’t have a library we can use.



# Cython

A Cython version - implemented in the file fastloop.pyx - looks something like this

So far all we’ve done is add some type declarations. For local variables we use the cdef keyword. For arrays we use ‘memoryviews’ which can accept numpy arrays as input.

Note that you don’t have to add type declarations in a *.pyx file. Any lines which use untyped variables will just remain in Python rather than being translated to C.

To compile we need a setup.py script, that looks something like this

then we compile from the terminal with



which generates a C code file fastloop.c and a compiled Python extension fastloop.so.

Lets test it out

from fastloop import rbf_network
%timeit rbf_network(X, beta, theta)

OK, but we can do a better. With Cython there are a few ‘tricks’ involved in achieving good performance. Here’s the first one, if we type this in the terminal

we generate a fastloop.html file which we can open in a browser


Lines highlighted yellow are still using Python and are slowing our code down. Our goal is get rid of yellow lines, especially any inside of loops.

Out first problem is that we’re still using the Python exponential function. We need to replace this with the C version. The main functions from math.h are included in the Cython libc library, so we just replace from math import exp with

Next we need to add some [compiler directives](http://docs.cython.org/src/reference/compilation.html#compiler-directives), the easiest way is to add this line to the top of the file

Note that with these checks turned off you can get segmentation faults rather than nice error messages, so its best to debug your code before putting this line in.

Next we can consider playing with compiler flags (these are C tricks rather than Cython tricks as such). When using gcc the most important option seems to be -ffast-math. From my limited experience, this can improve speeds a lot, with no noticeable loss of reliability. To implement these changes we need to modify our setup.py file

Now if we run cython fastloop.pyx -a again we will see the loops are now free of Python

The yellow outside the loops is irrelevant here (but would matter if we needed to call this function many times within another loop).

Now if we recompile and test it out we get

from fastloop import rbf_network
%timeit rbf_network(X, beta, theta)

OK, now we’re getting there.

# Calling C functions

So what else can we do? Well it turns out the exponential function is a bit of a bottleneck here, even the C version. One option is to use a [fast approximation to the exponential function](http://www.schraudolph.org/pubs/Schraudolph99.pdf)

From Cython its easy to call C code. Put the above code in vfastexp.h, then just add the following to our fastloop.pyx file

So now we can just use exp_approx() in place of exp(). This gives us

from fastloop import rbf_network
%timeit rbf_network(X, beta, theta)

# Cython inline

In [35]:
%%cython 
#%%cython -a 
from math import exp
import numpy as np

def rbf_network_0(X,beta,theta):

    N = X.shape[0]
    D = X.shape[1]
    Y = np.zeros(N)

    for i in range(N):
        for j in range(N):
            r = 0
            for d in range(D):
                r += (X[j, d] - X[i, d]) ** 2
            r = r**0.5
            Y[i] += beta[j] * exp(-(r * theta)**2)

    return Y

In [36]:
%timeit rbf_network_0(X, beta, theta)

1 loop, best of 3: 3.97 s per loop


We only get the speedups when we start optimizing the code by typing the variables, avoiding the use of the math lib, and omitting some bounds checking. This allows the function to be run in pure C and not touch the python interpreter.

In [37]:
%%cython 
#%%cython -a 
from math import exp
import numpy as np

def rbf_network_1(double[:, :] X,  double[:] beta, double theta):

    cdef int N = X.shape[0]
    cdef int D = X.shape[1]
    cdef double[:] Y = np.zeros(N)
    cdef int i, j, d
    cdef double r = 0

    for i in range(N):
        for j in range(N):
            r = 0
            for d in range(D):
                r += (X[j, d] - X[i, d]) ** 2
            r = r**0.5
            Y[i] += beta[j] * exp(-(r * theta)**2)

    return Y

In [38]:
%timeit rbf_network_1(X, beta, theta)

1 loop, best of 3: 167 ms per loop


In [21]:
%%cython -a
#from math import exp
from libc.math cimport exp

import numpy as np

def rbf_network_2(double[:, :] X,  double[:] beta, double theta):

    cdef int N = X.shape[0]
    cdef int D = X.shape[1]
    cdef double[:] Y = np.zeros(N)
    cdef int i, j, d
    cdef double r = 0

    for i in range(N):
        for j in range(N):
            r = 0
            for d in range(D):
                r += (X[j, d] - X[i, d]) ** 2
            r = r**0.5
            Y[i] += beta[j] * exp(-(r * theta)**2)

    return Y

In [39]:
%timeit rbf_network_2(X, beta, theta)

10 loops, best of 3: 99.4 ms per loop


In [40]:
%%cython --compile-args=-ffast-math --link-args=-ffast-math -a
# distutils: language=c

#from math import exp
from libc.math cimport exp
import numpy as np
import cython

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False)
def rbf_network_3(double[:, :] X,  double[:] beta, double theta):

    cdef int N = X.shape[0]
    cdef int D = X.shape[1]
    cdef double[:] Y = np.zeros(N)
    cdef int i, j, d
    cdef double r = 0

    for i in range(N):
        for j in range(N):
            r = 0
            for d in range(D):
                r += (X[j, d] - X[i, d]) ** 2
            r = r**0.5
            Y[i] += beta[j] * exp(-(r * theta)**2)

    return Y

In [41]:
%timeit rbf_network_3(X, beta, theta)

10 loops, best of 3: 30.8 ms per loop


And that's pretty good, but we're still only using a single CPU because of python's [Global Interpretter Lock (GIL)](http://stackoverflow.com/questions/1294382/what-is-a-global-interpreter-lock-gil). Let's use all those cores...

# Parallelisation in Cython

## Cython with OpenMP
Cython supports [parallel processing](http://docs.cython.org/src/userguide/parallelism.html) via threads using the OpenMP backend. What does that look like?

First of all, we have to add some extra compile flags to enable OpenMP. Next we put the loops in a nogil context which releases the GIL restriction, something we can only safely do when our code is in pure C without any interaction with Python objects.  <br>
Also in the same context block is parallel which sets up the OpenMP threading. Finally we see prange, parallel range, which executes the outer loop in parallel across the number of cores specified. <br>

Now to create a multi-threaded version of rbf_network we just need to replace range() in the first loop with the multi-treaded version prange() from cython.parallel. This tells the compiler to run the loop across multiple CPU cores. In this case, we have no concurrency problems: the order in which the loop is executed doesn’t matter. <br>

Notice the nogil argument in prange(N, nogil=True). In order to run multi-threaded code we need to turn off the GIL. This means that you can’t have Python code inside your multi-threaded loop or compilation will fail. It also means that any functions called inside the loop need to be defined nogil

In [42]:
%%cython --compile-args=-fopenmp --link-args=-fopenmp --compile-args=-ffast-math --link-args=-ffast-math --force -a

import cython
from cython.parallel import prange, parallel
import numpy as np
#from math import exp
from libc.math cimport exp

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False)
def rbf_network_multithread(double[:, :] X,  double[:] beta, double theta):

    cdef int N = X.shape[0]
    cdef int D = X.shape[1]
    cdef double[:] Y = np.zeros(N)
    cdef int i, j, d
    cdef double r = 0

    for i in prange(N, nogil=True):
        for j in range(N):
            r = 0
            for d in range(D):
                r += (X[j, d] - X[i, d])**2 
            r = r**0.5
            Y[i] += beta[j] * exp(-(r * theta)**2)

    return Y


In [43]:
D = 5
N = 1000
X = np.array([np.random.rand(N) for d in range(D)]).T
beta = np.random.rand(N)
theta = 10

In [44]:
%timeit rbf_network_multithread(X, beta, theta)

100 loops, best of 3: 11.6 ms per loop


# Conclusion

| Method        | Time (ms)           | Speed up Factor  |
| :------------- |:-------------:|:-----:|
| Python loops     | 4570 | - |
| Scipy     | 185      |  24  |
| Cython | 30.8      |  148   |
| Cython with parallelisation | 11.6      |  393   |

I was able to make this particular array operation roughly 400x faster using code that looks remarkably similar to the original naive python implementation. Also important to note that the function API did not change at all; the speed benefits and multithreading are completely transparent to the user of this function.

If you are processing an array using loops, you should definitely look at Cython, particularly Cython with OpenMP threading, to speed up your operations. Maybe not as easy as Python, but certainly much better than learning C.



This tutorial is a mix of some examples which can be found here:
+ [Fast Python loops](http://nealhughes.net/cython1/)
+ [Parallel computing in Cython - threads](http://nealhughes.net/parallelcomp2/)
+ [Parallelizing numpy array loops with Cython and OpenMP](http://www.perrygeo.com/parallelizing-numpy-array-loops-with-cython-and-mpi.html)